In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
import time
import pandas as pd

#Inspiration from https://github.com/arapfaik/scraping-glassdoor-selenium/blob/master/glassdoor%20scraping.ipynb

def get_jobs(keyword, num_jobs, verbose, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword
    
    #Initializing the webdriver
    driver = webdriver.Chrome()
    driver.set_window_size(1120, 1000)

    driver.get(url)
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            
            driver.find_element(By.CLASS_NAME, "selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element(By.XPATH, '//button[@class="e1jbctw80 ei0fd8p1 css-1n14mz9 e1q8sty40"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.XPATH, '//li[@data-test="jobListing"]')  #These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click() 
            time.sleep(10)
            
            try:
                company_name = driver.find_element(By.CSS_SELECTOR,'div.css-87uc0g.e1tk4kwz1').text
            except NoSuchElementException:
                company_name = -1 #You need to set a "not found value. It's important."
                
            try:
                location = driver.find_element(By.XPATH,'//div[@data-test="location"]').text
            except NoSuchElementException:
                location = -1
            
            try:
                job_title = driver.find_element(By.XPATH,'//div[@data-test="jobTitle"]').text
            except NoSuchElementException:
                job_title = -1
                
            try:
                salary_estimate = driver.find_element(By.XPATH,'//div[@class="css-1bluz6i e2u4hf13"]').text
            except NoSuchElementException:
                salary_estimate = -1 
            
            try:
                rating = driver.find_element(By.XPATH,'//div[@data-test="rating-info"]').text
            except NoSuchElementException:
                rating = -1 

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            try:
                driver.find_element(By.ID,'CompanyContainer').click()

                try:
                    size = driver.find_element(By.XPATH,'//span[text()="Size"]/following-sibling::span').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element(By.XPATH,'//span[text()="Founded"]/following-sibling::span').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element(By.XPATH,'//span[text()="Type"]/following-sibling::span').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element(By.XPATH,'//span[text()="Industry"]/following-sibling::span').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element(By.XPATH,'//span[text()="Sector"]/following-sibling::span').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element(By.XPATH,'//span[text()="Sector"]/following-sibling::span').text
                except NoSuchElementException:
                    revenue = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, '//button[@data-test="pagination-next"]').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.



In [ ]:
df = get_jobs('data scientist', 1000, False, 20) #Scrape 1000 jobs from glassdoor

In [3]:
df.to_csv('jobs.csv', index = False) #Convert to csv